In [1]:
import pandas as pd

import time

import os

from datetime import datetime

import numpy as np

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.palettes import Set1_5
output_notebook()

Loading BokehJS ...

In [2]:
src_folder = '/home/narsi/Documents/CSEE5690_data/A2_WEB_SYS_LOG/curated_logs'

In [3]:
def get_filepaths(directory):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.

In [4]:
file_path = get_filepaths(src_folder)

In [5]:
final_log = pd.DataFrame({'url' : [],'count' : [],'date' : []})
unique_days = []
for F in file_path:
    log_data = pd.read_csv(F,',',header=None, error_bad_lines=False)
    x = pd.Series.value_counts(log_data[4])
    urls = []
    count = np.asarray(x)
    date = []
    
    unique_days.append(log_data[1][0])
    
    for i in range(len(x)):
        date.append(log_data[1][0])
        urls.append(x.axes[0][i])
    dummy = pd.DataFrame({'url' : urls,'count' : count,'date' : date})
    final_log = final_log.append(dummy)

In [6]:
unique_urls = np.asarray(pd.Series.value_counts(final_log['url']).axes[0])
unique_date = pd.DataFrame({'date':pd.to_datetime(unique_days)}).sort_values(by='date')
final_log['date'] = pd.to_datetime(final_log['date'])
url_by_date_count = np.zeros((len(unique_urls),len(unique_date)))
for i in range(len(unique_urls)):
    for j in range(len(unique_date)):
        x = np.asarray(final_log['count'][(final_log['date'] == unique_date['date'][j]) & (final_log['url'] == unique_urls[i])])
        if len(x) == 0:
            url_by_date_count[i][j] = 0
        else:
            url_by_date_count[i][j] = x


In [7]:
mean_count = np.mean(url_by_date_count,axis=1)
top_5_urls = np.argsort(mean_count) # Sort in ascending order
top_5_urls = top_5_urls[-5:]

In [8]:
def datetime(x):
    return np.array(x['date'], dtype=np.datetime64)

line_type = ['solid', 'dashed', 'dotted']

p1 = figure(x_axis_type="datetime", title="Top-5 URL'S")
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Count'

for i in range(1,4):
    p1.line(datetime(unique_date),url_by_date_count[top_5_urls[-1 * i]], legend=unique_urls[top_5_urls[-1 * i]],color = Set1_5[i], line_width=i+1,line_dash = line_type[i-1])
p1.legend.location = "top_left"
show(p1)

In [31]:
File_name_list = []
date_list = []
for F in file_path:
    X =  F.split('/') # Spliting the src file path to get only the file name at the end
    file_name = X[-1].split('.')
    date_name = file_name[0].split('_')
    date_time = datetime.strptime(date_name[1]+date_name[2], ' %b %d %Y').strftime('%Y-%m-%d')
    File_name_list.append(X[-1])
    date_list.append(date_time)

In [35]:
pd.DataFrame({'File Name':File_name_list,'Date':date_list}).sort_values(by='Date').to_csv('dates_order.csv', header=False, index=False)